In [3]:
from fastai.text import *
from fastai.lm_rnn import *
import sentencepiece as sp

In [4]:
BTW17 = Path("../data/btw17")
germeval2017  = Path("../data/germeval2017")
WORK = Path("../work/")
!ls {WORK}
!ls {germeval2017}/

btw-nouniq30k  ge2017  shared  shared-wiki  wiki30k  wikige2017
dev_v1.4.tsv  test_TIMESTAMP1.tsv  test_TIMESTAMP2.tsv	train_v1.4.tsv
dev_v1.4.xml  test_TIMESTAMP1.xml  test_TIMESTAMP2.xml	train_v1.4.xml


In [5]:
#wiki=Path("../data/wiki/de")
#!ls {wiki}

train.csv  train.txt  val.csv


In [6]:
#train_wiki = pd.read_csv(wiki/"train.csv", header=None)
#np.savetxt(wiki/'train.txt', train_wiki.values, fmt="%s")
#!cat {BTW17}/text.txt >> {wiki}/train.txt

In [7]:
def read_germeval(csvfn):
    cols=["id", "text", "relevance", "sentiment", "aspect:polarity"]
    df = pd.read_csv(germeval2017/csvfn , delimiter="\t", header=None, names=cols, index_col=False)
    df.sentiment = pd.Categorical(df.sentiment, categories=['neutral', 'negative', 'positive'])
    df.relevance = pd.Categorical(df.relevance, categories=[False, True])
    sen = pd.DataFrame(df.text)
    sen['sentiment'] = df.sentiment.cat.codes
    rel = pd.DataFrame(df.text)
    rel['relevance'] = df.relevance.cat.codes
    return sen,rel

# sen_trn,_ = read_germeval('train_v1.4.tsv')
# sen_val,_ = read_germeval('dev_v1.4.tsv')
# sen_test1,_ = read_germeval('test_TIMESTAMP1.tsv')
# sen_test2,_ = read_germeval('test_TIMESTAMP2.tsv')


In [8]:
sen,_ = read_germeval('dev_v1.4.tsv')


In [9]:
import concurrent.futures
spp = sp.SentencePieceProcessor()
p = WORK/'btw-nouniq30k'
spp.Load(str( p/ 'tmp' / 'sp.model'))
vs = spp.GetPieceSize() #len(itos)
spp.SetEncodeExtraOptions("bos:eos")
def toIds_raw(l):
    return spp.EncodeAsIds(str(l))   # to cover for NaN like on line 164

def tokenize(lines):
    WORKERS=num_cpus()
    results=[]
    chunksize=max(1,int(len(lines)/100/WORKERS))
    with concurrent.futures.ProcessPoolExecutor() as e:
        for result in e.map(toIds_raw, lines, chunksize=chunksize):
            results.append(np.array(result))
    return np.array(results)

In [10]:
sen['tok'] = tokenize(sen['text'])
sen['tok'][2]

array([    2,     7,   232,   726,  2608,     7,  2856,  1738,  2323,   972,     6, 15487,    38,   329,
          22,  4367, 14644,    23, 14990,  2761,     6,  2348,    12,    18,  3545,     5,  2446,     5,
        6729,     5,   392, 21634,   108,     9,   267,    14,    10,     5,    15,    16,  6510,  7137,
        6786,   119,    85,   254,    79,     3])

In [11]:
# trn, val, test
germeval2017_tmp = WORK/'ge2017'/'tmp'
germeval2017_tmp.mkdir(exist_ok=True, parents=True)
def convert_germeval(fn, set_name):
    sen,_ = read_germeval(fn)
    sen['tok'] = tokenize(sen['text'])
    np.save(germeval2017_tmp/f'{set_name}_ids.npy' ,np.array(sen['tok']))
    np.save(germeval2017_tmp/f'lbl_{set_name}.npy' ,np.array(sen['sentiment']))
    return sen['tok']
# 'tmp/val_{IDS}_bwd.npy'
# 'tmp/trn_{IDS}.npy'
# 'tmp/val_{IDS}.npy'
# 'tmp/lbl_trn{train_file_id}.npy'
# 'tmp/lbl_val.npy'

In [12]:
# sen_trn,_ = read_germeval('train_v1.4.tsv')
# sen_val,_ = read_germeval('dev_v1.4.tsv')
# sen_test1,_ = read_germeval('test_TIMESTAMP1.tsv')
# sen_test2,_ = read_germeval('test_TIMESTAMP2.tsv'

In [13]:
all_text = [[]]*4

In [14]:
all_text[0] = convert_germeval('train_v1.4.tsv', 'trn')

In [15]:
all_text[1] =  [] # do not add val set to all text
val = convert_germeval('dev_v1.4.tsv', 'val')

In [16]:
cols=["id", "text", "relevance", "sentiment", "aspect:polarity"]
df = pd.read_csv(germeval2017/'test_TIMESTAMP1.tsv' , delimiter="\t", header=None, names=cols)

In [17]:
all_text[2] = convert_germeval('test_TIMESTAMP1.tsv', 'test1')

In [18]:
all_text[3] = convert_germeval('test_TIMESTAMP2.tsv', 'test2')

In [19]:
[np.array(a).shape for a in all_text]

[(20941,), (0,), (2566,), (1842,)]

In [160]:
all_text_np = np.concatenate(all_text)

In [161]:
np.save(germeval2017_tmp/f'trn_ids_all.npy' ,all_text_np)

In [91]:
! head -n 165 {germeval2017}/dev_v1.4.tsv | tail -n 1

https://plus.google.com/106967535142671617878/posts/5x5NBQhH2Pv	null	false	neutral


# Sentence piece statistics

In [20]:
sum([len(s) for s in val])

357133

In [21]:
print("Maximal length of sentence in tokens", max([len(s) for s in val]))
print("AVG Length in tokens", sum([len(s) for s in val])/len(val))
print("Length of valuation set", len(val))

Maximal length of sentence in tokens 6865
AVG Length in tokens 138.20936532507739
Length of valuation set 2584


In [176]:
val_t,_ = read_germeval('dev_v1.4.tsv')

In [187]:
max([(len(str(s).split()), idx, s) for idx,s in enumerate(val_t['text'])])

(4475,
 20,
 '+++ Newsticker FlÃ¼chtlingskrise +++: Deutschland will mit Kriegsschiffen Jagd auf Schleuser machen +++ 21.23 Uhr: Polizeibekannter Neonazi bewachte Flüchtlinge +++ Großbritannien mehr Flüchtlinge aufnehmen. Dazu sollen Tausende Syrer direkt aus den Flüchtlingslagern ausgewählt werden. +++ 20.43 Uhr: Regierung will Mandat für bewaffneten Einsatz gegen Schlepper +++ Die EU-Außenminister kommen am Freitag in Luxemburg zusammen, um unter anderem über die dramatische Situation der Flüchtlinge zu beraten. Schwerpunkt der zweitägigen Gespräche soll die Situation in den Herkunfts- und Transitstaaten sein. Deutschland und Frankreich starteten kurz vor dem Treffen eine gemeinsame Initiative für verbindliche Aufnahmequoten in der EU. In Ungarn herrschten wegen der großen Zahl von Flüchtlingen zuletzt teilweise chaotische Zustände. In der Stadt Bicske wehrten sich am Abend rund 500 Menschen gegen den Transport in ein Flüchtlingslager. Sie waren an der Weiterreise per Zug nach Westen

In [191]:
! head -n 21 {germeval2017}/dev_v1.4.tsv | tail -n 1

http://nachedeu.com/newsticker-fla%c2%bcchtlingskrise-deutschland-will-mit-kriegsschiffen-jagd-auf-schleuser-machen/	+++ Newsticker FlÃ¼chtlingskrise +++: Deutschland will mit Kriegsschiffen Jagd auf Schleuser machen +++ 21.23 Uhr: Polizeibekannter Neonazi bewachte Flüchtlinge +++ Großbritannien mehr Flüchtlinge aufnehmen. Dazu sollen Tausende Syrer direkt aus den Flüchtlingslagern ausgewählt werden. +++ 20.43 Uhr: Regierung will Mandat für bewaffneten Einsatz gegen Schlepper +++ Die EU-Außenminister kommen am Freitag in Luxemburg zusammen, um unter anderem über die dramatische Situation der Flüchtlinge zu beraten. Schwerpunkt der zweitägigen Gespräche soll die Situation in den Herkunfts- und Transitstaaten sein. Deutschland und Frankreich starteten kurz vor dem Treffen eine gemeinsame Initiative für verbindliche Aufnahmequoten in der EU. In Ungarn herrschten wegen der großen Zahl von Flüchtlingen zuletzt teilweise chaotische Zustände. In der Stadt Bicske wehrten sich am Abend rund 500

In [16]:
len (lines)

1213131

In [168]:
valbtw17 = np.load(p/'tmp'/'val_ids.npy')

In [171]:
len(valbtw17)

121484

In [172]:
max([len(s) for s in valbtw17])

112

In [173]:
sum([len(s) for s in valbtw17])

3465393

In [193]:
lblbtw17 = np.load(germeval2017_tmp/'lbl_val.npy')

In [194]:
lblbtw17.shape

(2584,)

In [197]:
lblbtw17.squeeze()[:,None][:,None].shape

(2584, 1, 1)

In [25]:
spp = sp.SentencePieceProcessor()
spp.Load(str( p/ 'tmp' / 'sp.model'))
vs = spp.GetPieceSize() #len(itos)
vs
#tokens_fraction = float(len(val_lm)) / (len(spp.DecodeIds(val_lm.tolist()).split()) + (val_lm == EOS_ID).sum())

30000

In [192]:
spp.IdToPiece(0),spp.IdToPiece(1),spp.IdToPiece(2),spp.IdToPiece(3),

('<unk>', '<pad>', '<s>', '</s>')

In [37]:
spp.SetEncodeExtraOptions("bos:eos")
[spp.EncodeAsIds(l) for l in tqdm(lines[:1])]

100%|██████████| 1/1 [00:00<00:00, 1552.30it/s]


[[2,
  4,
  196,
  11,
  194,
  3257,
  12,
  137,
  829,
  9,
  1452,
  923,
  628,
  503,
  13,
  14,
  10,
  5,
  15,
  16,
  5439,
  7551,
  77,
  9006,
  10080,
  88,
  3]]